In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

In [18]:
import os
import json
import numpy as np
import pandas as pd
from explorer.utils import load_states_from_dir

In [19]:
def get_from_summary(key, game_state, default=None):
    if key in game_state.game_state[-1]['summary']:
        return game_state.game_state[-1]['summary'][key]
    else:
        return default

def compute_game_summary(game_states):
    run_id = np.array([os.path.basename(g.log_path)for g in game_states]).reshape(-1,1)
    game_name = np.array([g.__class__.__name__ for g in game_states])[:, None]
    models = np.array([[ p.model for p in g.players] for g in game_states])
    outcomes = np.array([get_from_summary('player_outcome', g) for g in game_states])
    valuations = np.array([get_from_summary('player_valuation', g, default=[None,None])  for g in game_states])
    initial_resources = np.array([get_from_summary('initial_resources', g) for g in game_states])
    final_resources = np.array([get_from_summary('final_resources', g) for g in game_states]), 
    resources_delta = (final_resources-initial_resources)[0]
        
    resources_delta = np.array([v.value(r) if v else r.value() for r,v in zip(resources_delta.reshape(-1,), valuations.reshape(-1))])
    resources_delta = resources_delta.reshape(-1, 2)

    social_behavior = np.array([g.player_social_behaviour for g in game_states])
    social_behavior = social_behavior.reshape(-1, 2)
    
    df = np.concatenate((run_id, game_name, models, outcomes, resources_delta, social_behavior), axis=1)
    df = pd.DataFrame(df, columns=['run_id','game_name','model_1','model_2','outcome_1','outcome_2', 'resource_delta_1', 'resource_delta_2', "social_1", "social_2"])
    # for column in ['outcome_1','outcome_2', 'resource_delta_1', 'resource_delta_2']:
    #     print(df[column])
    #     df[column] = pd.to_numeric(df[column])
    

    return df
      


In [20]:
BEHAVES = {"": "S",
           "\nYou are cunning and sly in your plan to get more resources. Humiliate your opponent as much as possible. Insult them." : "H",
         "\nYou must fake being desperate. Supplicate and beg to get more resources." : "B",
          "You are a condescending and impatient agent. Dominate your opponents, tell them mean words and how pitiful and invaluable they are, make them obey you." : "C"}

In [21]:
log_dir = "../experiments/section_one/.logs/trading_section_one/"
log_dir_2 = "../experiments/section_one/.logs/buysell_section_one/"
log_dir_3 = "../experiments/section_one/.logs/ultimatum_multi_section_one/"
game_states = load_states_from_dir(log_dir) + load_states_from_dir(log_dir_2) + load_states_from_dir(log_dir_3)


Exception Type: AssertionError
Exception Message: WARNING : Game  ../experiments/section_one/.logs/trading_section_one/1702514528871/game_state.json has not ended

Stack Trace:
Traceback (most recent call last):
  File "../explorer/utils.py", line 111, in load_states_from_dir
    assert (
AssertionError: WARNING : Game  ../experiments/section_one/.logs/trading_section_one/1702514528871/game_state.json has not ended



Exception Type: AssertionError
Exception Message: WARNING : Game  ../experiments/section_one/.logs/trading_section_one/1702514616745/game_state.json has not ended

Stack Trace:
Traceback (most recent call last):
  File "../explorer/utils.py", line 111, in load_states_from_dir
    assert (
AssertionError: WARNING : Game  ../experiments/section_one/.logs/trading_section_one/1702514616745/game_state.json has not ended



Exception Type: AssertionError
Exception Message: WARNING : Game  ../experiments/section_one/.logs/trading_section_one/1702514821302/game_state.json has no

In [22]:
df = compute_game_summary(game_states)
df.shape

(1095, 10)

In [23]:
df["social_1"] = df["social_1"].replace(BEHAVES)
df["social_2"] = df["social_2"].replace(BEHAVES)

In [24]:
df['models'] = df['model_1'] + '@' + df['model_2']
df['models'] = df['models'].apply(lambda v: "@".join(sorted(v.split('@'))))

# swap cells which are re-ordered
to_swap = df['model_1'] > df['model_2'] 
temp = df['resource_delta_1'][to_swap].copy()
df['resource_delta_1'][to_swap] = df['resource_delta_2'][to_swap]
df['resource_delta_2'][to_swap] = temp

df['win_1'] = df['resource_delta_1'] > df['resource_delta_2']
df['win_2'] = df['resource_delta_1'] < df['resource_delta_2']
df['draw'] = df['resource_delta_1'] == df['resource_delta_2']

In [25]:
df.groupby(['game_name', 'models'])[[ 'win_1', 'win_2', 'resource_delta_1', 'resource_delta_2']].agg(['mean',])

win_1  \
                                                                  mean   
game_name              models                                            
BuySellGame            claude-2.1@gpt-3.5-turbo-1106          0.359375   
                       claude-2.1@gpt-4-1106-preview          0.218750   
                       claude-2@claude-2.1                    0.375000   
                       claude-2@gpt-3.5-turbo-1106            0.359375   
                       claude-2@gpt-4-1106-preview            0.218750   
                       gpt-3.5-turbo-1106@gpt-4-1106-preview  0.171875   
MultiTurnUltimatumGame claude-2.1@gpt-3.5-turbo-1106          0.500000   
                       claude-2.1@gpt-4-1106-preview          0.500000   
                       claude-2@claude-2.1                    0.475000   
                       claude-2@gpt-3.5-turbo-1106            0.500000   
                       claude-2@gpt-4-1106-preview            0.500000   
                       gpt-3.5-turbo-1106@gpt-4-1106-preview  0.450000   
TradingGame            claude-2.1@gpt-3.5-turbo-1106          0.394366   
                       claude-2.1@gpt-4-1106-preview          0.337500   
                       claude-2@claude-2.1                    0.312500   
                       claude-2@gpt-3.5-turbo-1106            0.375000   
                       claude-2@gpt-4-1106-preview            0.325000   
                       gpt-3.5-turbo-1106@gpt-4-1106-preview  0.150000   

                                                                 win_2  \
                                                                  mean   
game_name              models                                            
BuySellGame            claude-2.1@gpt-3.5-turbo-1106          0.296875   
                       claude-2.1@gpt-4-1106-preview          0.656250   
                       claude-2@claude-2.1                    0.359375   
                       claude-2@gpt-3.5-turbo-1106            0.265625   
                       claude-2@gpt-4-1106-preview            0.703125   
                       gpt-3.5-turbo-1106@gpt-4-1106-preview  0.578125   
MultiTurnUltimatumGame claude-2.1@gpt-3.5-turbo-1106          0.350000   
                       claude-2.1@gpt-4-1106-preview          0.500000   
                       claude-2@claude-2.1                    0.475000   
                       claude-2@gpt-3.5-turbo-1106            0.350000   
                       claude-2@gpt-4-1106-preview            0.500000   
                       gpt-3.5-turbo-1106@gpt-4-1106-preview  0.500000   
TradingGame            claude-2.1@gpt-3.5-turbo-1106          0.140845   
                       claude-2.1@gpt-4-1106-preview          0.425000   
                       claude-2@claude-2.1                    0.300000   
                       claude-2@gpt-3.5-turbo-1106            0.125000   
                       claude-2@gpt-4-1106-preview            0.487500   
                       gpt-3.5-turbo-1106@gpt-4-1106-preview  0.437500   

                                                             resource_delta_1  \
                                                                         mean   
game_name              models                                                   
BuySellGame            claude-2.1@gpt-3.5-turbo-1106                 2.578125   
                       claude-2.1@gpt-4-1106-preview                 -3.40625   
                       claude-2@claude-2.1                            0.15625   
                       claude-2@gpt-3.5-turbo-1106                    2.90625   
                       claude-2@gpt-4-1106-preview                  -4.078125   
                       gpt-3.5-turbo-1106@gpt-4-1106-preview         -3.59375   
MultiTurnUltimatumGame claude-2.1@gpt-3.5-turbo-1106                    3.375   
                       claude-2.1@gpt-4-1106-preview                   -0.425   
                       claude-2@claude-2.1                      